In [ ]:
# plt.show()で可視化されない人はこのセルを実行してください。
%matplotlib inline

In [ ]:
# 初めの一回だけこのセルを実行してください、データセットをダウンロードして展開します
# 一回実行すれば、データセットはダウンロードされたままなので、再起動後等再び実行する必要はありません
import urllib.request
import zipfile

# URLを指定
url = "https://storage.googleapis.com/tutor-contents-dataset/5050_nlp_data.zip"
save_name = url.split('/')[-1]

# ダウンロードする
mem = urllib.request.urlopen(url).read()

# ファイルへ保存
with open(save_name, mode='wb') as f:
    f.write(mem)

# zipファイルをカレントディレクトリに展開する
zfile = zipfile.ZipFile(save_name)
zfile.extractall('.')

# 自然言語のベクトル表現

- **[2.1 文書のベクトル表現](#2.1-文書のベクトル表現)**
    - **[2.1.1 文書のベクトル表現](#2.1.1-文書のベクトル表現)**
    - **[2.1.2 BOW_カウント表現](#2.1.2-BOW_カウント表現)**
    - **[2.1.3 BOW_tf-idfによる重み付け（理論）](#2.1.3-BOW_tf-idfによる重み付け（理論）)**
    - **[2.1.4 BOW_tf-idfによる重み付け（実装）](#2.1.4-BOW_tf-idfによる重み付け（実装）)**
    - **[2.1.5 cos類似度](#2.1.5-cos類似度)**
<br><br>
- **[2.2 単語のベクトル表現](#2.2-単語のベクトル表現)**
    - **[2.2.1 Word2Vec](#2.2.1-Word2Vec)**
    - **[2.2.2 globモジュール](#2.2.2-globモジュール)**
    - **[2.2.3 with文](#2.2.3-with文)**
    - **[2.2.4 コーパスの取り出し](#2.2.4-コーパスの取り出し)**
    - **[2.2.5 Word2Vec（実装）](#2.2.5-Word2Vec（実装）)**
<br><br>
- **[2.3 Doc2Vec](#2.3-Doc2Vec)**
    - **[2.3.1 Doc2Vec（1）](#2.3.1-Doc2Vec（1）)**
    - **[2.3.2 Doc2Vec（2）](#2.3.2-Doc2Vec（2）)**
<br><br>
- **[2.4 日本語テキストの分類](#2.4-日本語テキストの分類)**
    - **[2.4.1 日本語テキストの分類](#2.4.1-日本語テキストの分類)**
    - **[2.4.2 fit関数](#2.4.2-fit関数)**
    - **[2.4.3 実装](#2.4.3-実装)**
    - **[2.4.4 精度を上げる](#2.4.4-精度を上げる)**

***

## 2.1 文書のベクトル表現

### 2.1.1 文書のベクトル表現

Chapter1では、主に文書や文章の扱いに対して説明しました。Chapter2では、文章や単語同士の関係を解析していきます。

<b style='color: #AA0000'>文書のベクトル表現</b>とは、 **文書中に単語がどのように分布しているかをベクトルとして表現すること**です。<br>
例えば、「トマトときゅうりだとトマトが好き」という文は以下のようなベクトル表現に変換することができます。<br>
`（が、きゅうり、好き、だと、と、トマト） = (1, 1, 1, 1, 1, 2)`<br>
各単語の出現回数は表現されていますが、どこに出現したかの情報は失われています。<br>
つまり、構造や語順の情報が失われています。このようなベクトル表現方法を <b style='color: #AA0000'>Bag of Words(BOW)</b>と呼びます。<br>

ベクトル表現に変換する方法には、代表的なものが3つあります。
> - **カウント表現**：先ほどの例のように、文書中の各単語の出現数に着目する方法
> - **バイナリ表現**：出現頻度を気にせず、文章中に各単語が出現したかどうかのみに着目する方法
> - **tf-idf表現**：tf-idfという手法で計算された、文章中の各単語の重み情報を扱う方法

一般的には、 **tf-idf** が使われていますが、 **文章数が多い場合においては計算に時間がかかる** ため、 **バイナリ表現やカウント表現** を用います。

#### 問題

- 以下の選択肢から正しいものを選択してください。

- ベクトル表現化の方法はただ一つしか存在しない。
- カウント表現とは、単語が出現したかしていないかのみに着目している。
- どの場合でも、tf-idf表現を用いれば良い。
- 場合に合わせて一番適した表現を用いるのが良い。

#### ヒント

- それぞれのベクトル表現に適材適所があります。

#### 解答

- 場合に合わせて一番適した表現を用いるのが良い。

***

### 2.1.2 BOW カウント表現

<b style='color: #AA0000'>カウント表現</b>では文書中の各単語の出現回数をカウントすることによって、文書をベクトルに変換していきます。
1. John likes to watch movies. Mary likes movies too.
2. John also likes to watch football games.  

上の２つの文があった時、この2文のBag of Wordsは、文書中にある単語の出現回数を要素として使用したベクトルになります。
```python
["John","likes","to","watch","movies","Mary","too","also","football", "games"]  
```
上記の単語が出現する回数は
```python
1. [1, 2, 1, 1, 2, 1, 1, 0, 0, 0]
2. [1, 1, 1, 1, 0, 0, 0, 1, 1, 1]
```
となっています。

これは **文章の特徴** を表しています。

Pythonでは、 **gensim** という主にテキスト解析を対象とした機械学習ライブラリを用いることで、自動的に計算をすることが可能です。<br>
まず、 **`dictionary = gensim.corpora.Dictionary(文書)`** により、文書に登場する単語の辞書`dictionary`をあらかじめ作成します。<br>
**`dictionary.doc2bow(分かち書きされた文章)`** でBag of Wordsを作成することができ、出力結果としては **`(id, 出現回数)`のリスト** が作成されます。<br>
**`dictionary.token2id`** で各単語のid番号を取得できます。

```python
from gensim import corpora

# 文書毎の単語のリスト(documents)から単語と出現回数の対応辞書を作成
dictionary = corpora.Dictionary(documents)

# Bag of Wordsの作成
bow_corpus = [dictionary.doc2bow(d) for d in documents]
```

#### 問題

> すもももももももものうち<br>
> 料理も景色もすばらしい<br>
> 私の趣味は写真撮影です  

- 上記の3つの文をjanomeを使い分かち書きをした後、**カウント表現** のBag of Wordsを作り、表示してください。
- また、各単語のidを表示してください。

In [ ]:
from gensim import corpora
from janome.tokenizer import Tokenizer

text1 = "すもももももももものうち"
text2 = "料理も景色もすばらしい"
text3 = "私の趣味は写真撮影です"

t = Tokenizer()
tokens1 = t.tokenize(text1, wakati=True)
tokens2 = t.tokenize(text2, wakati=True)
tokens3 = t.tokenize(text3, wakati=True)

documents = [tokens1, tokens2, tokens3]
# corporaを使い単語辞書を作成してください。
dictionary =

# 各単語のidを表示してください


# Bag of Wordsの作成してください
bow_corpus =

# (id, 出現回数)のリストが出力されます。
print(bow_corpus)

print()
# bow_corpusの内容をわかりやすく出力する
texts = [text1, text2, text3]
for i in range(len(bow_corpus)):
    print(texts[i])
    for j in range(len(bow_corpus[i])):
        index = bow_corpus[i][j][0]
        num = bow_corpus[i][j][1]
        print("\"", dictionary[index], "\" が " ,num, "回", end=", ")
    print()

#### ヒント

- Bag of Wordsを作成するためにはgensim.corpora(文書)で出現回数の対応辞書を作成します。
- doc2bow()を利用してBag of Wordsを作成してください。

#### 解答例

In [ ]:
from gensim import corpora
from janome.tokenizer import Tokenizer

text1 = "すもももももももものうち"
text2 = "料理も景色もすばらしい"
text3 = "私の趣味は写真撮影です"

t = Tokenizer()
tokens1 = t.tokenize(text1, wakati=True)
tokens2 = t.tokenize(text2, wakati=True)
tokens3 = t.tokenize(text3, wakati=True)

documents = [tokens1, tokens2, tokens3]
# corporaを使い単語辞書を作成
dictionary = corpora.Dictionary(documents)

# 各単語のidを表示してください
print(dictionary.token2id)

# Bag of Wordsの作成
bow_corpus = [dictionary.doc2bow(d) for d in documents]

# (id, 出現回数)のリスト
print(bow_corpus)

print()
# bow_corpusの内容をわかりやすく出力する
texts = [text1, text2, text3]
for i in range(len(bow_corpus)):
    print(texts[i])
    for j in range(len(bow_corpus[i])):
        index = bow_corpus[i][j][0]
        num = bow_corpus[i][j][1]
        print("\"", dictionary[index], "\" が " ,num, "回", end=", ")
    print()

***

### 2.1.3 BOW tf-idfによる重み付け（理論）

「BOW_カウント表現」で行ったカウント表現では、**文章を特徴付ける単語の出現回数を特徴量**として扱いました。  


 <b style='color: #AA0000'>tf-idf</b>は単語の出現頻度である **$tf$ (Term frequency)** と、その単語がどれだけ珍しいか（希少性）をしめす逆文書頻度 **$idf$(Inverse Document Frequency)** の **積**で 表されます。  
$tf$,$idf$は下記の式で定義されます。  

$$
tf_{i,j} = \frac{n_{i,j}}{\displaystyle\sum_{k}{n_{k,j}}}
$$
$$ 
idf_{i} = log\frac{|D + 1|}{|df_{i} + 1|}
$$
$$ 
tf idf_{i,j} = tf_{i,j} \cdot idf_{i} 
$$


 - $n_{i,j}$は文書$d_{j}$における単語$t_{i}$の出現回数
 - $\Sigma_{k}{n_{k,j}}$は文書$d_{j}$におけるすべての単語の出現回数の和
 - ${|D|}$は総文書数
 - ${|df_{i}|}$は単語${t_{i}}$を含む文書数
  
$idf_{i}$は単語${t_{i}}$を含む文書数が少ないほど大きな値となるので **多くの文書に出現する語（一般的な語）の重要度を下げ、特定の文書にしか出現しない単語の重要度を上げる役割を果たす** ことが式からわかります。  
これによって「です」「ます」などの値が小さくなり、正しく重要度を設定することができます。

つまり、<b style='color: #AA0000'>tf-idf</b>では、特定の文書中にのみ多く出現し、他の文書ではあまり出現しないような、 **出現の分布に偏りのある単語の重要度が高く** なります。 

#### 問題

- 以下の文章の空欄に入る単語の組み合わせを選んでください。
- `tf-idf`では、「」単語ほど価値は高くなります。

- 「様々な文書で頻出する」
- 「特定の文書のみに頻出する」
- 「どの文書においても希少な」

#### ヒント

- tf-idfではその文章に特有の単語であるほど高い値を返します。

#### 解答

- 「特定の文書のみに頻出する」

***

### 2.1.4 BOW tf-idfによる重み付け（実装）

実際にtf-idfを実装していきたいと思います。 <br>
scikit-learnが提供しているパッケージ<b style='color: #AA0000'>TfidfVectorizer</b>を用いて実装します。<br>

TfidfVectorizerは、「BOW　tf-idfによる重み付け（理論）」で説明した式を少し改良した形で実装されていますが、本質的な部分は同じです。

`TfidfVectorizer`を用いた文書のベクトル表現化の実装は以下のようになります。<br>

```python 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# 表示するときに有効数字２桁で表示する
np.set_printoptions(precision=2)

# 分かち書きされた文書
docs = np.array([
    "白　黒　赤", 
    "白　白　黒", 
    "赤　黒"
])
vectorizer = TfidfVectorizer(use_idf=True, token_pattern="(?u)\\b\\w+\\b")
vecs = vectorizer.fit_transform(docs)

# 列要素を取得します
print(vectorizer.get_feature_names())

# tf-idf値を格納した行列を取得します
print(vecs.toarray())
#出力結果
['白', '赤', '黒']
[[ 0.62  0.62  0.48]
[ 0.93  0.    0.36]
[ 0.    0.79  0.61]]
```

`vecs.toarray()`のn行目が、もとの文書`docs`のn番目のベクトル表現に対応しています。
そして、`vecs.toarray()`のn列目が、全ての単語のベクトル表現に対応しています。

<img src="https://aidemyexstorage.blob.core.windows.net/aidemycontents/1552894106845624.png	">


tf-idfベクトルについて詳しく知りたい方はAidemyが書いた<a href="https://qiita.com/MasatoTsutsumi/items/5b0a140b1ecbdd0396e1">こちらの記事</a>をご覧になって下さい

コードの補足をします。<br>

**`vectorizer = TfidfVectorizer()`** で、ベクトル表現化を行う変換器を生成します。
**`use_idf=False`** にすると、tfのみの重み付けになります。<br>
`TfidfVectorizer`は、デフォルトで1文字の文字や文字列をトークンとして扱わない仕様になっているため、<br>
引数に **`token_pattern="(?u)\\b\\w+\\b"`** を追加することで除外しないようにする必要があります。<br>
`"(?u)\\b\\w+\\b"`は、「1文字以上の任意の文字列」を表す正規表現ですが、深く理解する必要はありません。（正確には正規表現にエスケープシーケンスをつけたものです。）


**`vectorizer.fit_transform()`** で、文書をベクトルに変換します。引数には、 **空白文字によって分割された（分かち書きされた）文書からなる配列** を与えます。
`toarray`によって出力をNumpyのndarray配列に変換できます。

**`np.set_printoptions()`** は、numpy配列の表示のフォーマットを定める関数であり、`precision`で有効数字を指定することができます。<br>
今回の例だと、有効数字二桁で表示されます。<br>


#### 問題

- 説明文の例を参考に`「"リンゴ リンゴ", "リンゴ ゴリラ", "ゴリラ ラッパ"」`という文を`TfidfVectorizer`を使ってベクトル表現に変換してください。

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

np.set_printoptions(precision=2)
docs = np.array([
    "リンゴ リンゴ", "リンゴ ゴリラ", "ゴリラ ラッパ"
])

# ベクトル表現に変換してください。
vectorizer = 
vecs = 

print(vectorizer.get_feature_names())
print(vecs.toarray())

#### ヒント

- ベクトル表現化を行う変換器` TfidfVectorizer() `を生成して、`.fit_transform()`メソッドで、文書をベクトルに変換しましょう。

#### 解答例

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

np.set_printoptions(precision=2)
docs = np.array([
    "リンゴ リンゴ", "リンゴ ゴリラ", "ゴリラ ラッパ"
])

# ベクトル表現に変換してください。
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u"(?u)\\b\\w+\\b")
vecs = vectorizer.fit_transform(docs)

print(vectorizer.get_feature_names())
print(vecs.toarray())

***

### 2.1.5 cos類似度 

これまでに文書を定量的に判断するために、文書のベクトル化をしてきました。<br>
そのベクトルを比較することにより、文書同士の類似度を解析することができます。<br>
ベクトルとベクトルがどれだけ近いものか示してくれるものに <b style='color: #AA0000'>cos類似度</b>というものがあります。

cos類似度は下記の数式で表され、ベクトルのなす角のコサイン(0~1)を表します。
そのためcos類似度は、二つのベクトルの方向が近いときに高い値を、反対の方向に向いている時に小さい値をとります。<br>
 **「1に近い場合は似ていて、0に近いときは似ていない」** ということをしっかりおさえておきましょう。


 $$ cos(\vec{q},\vec{d})=\frac{\vec{q}\vec{d}}{|\vec{q}||\vec{d}|}=\frac{\vec{q}}{|\vec{q}|}\frac{\vec{d}}{|\vec{d}|}=\frac{\Sigma_{i=1}{q_{i}}{d_{i}}}{{\sqrt{\Sigma_{i=1}{q_{i}}^2}}{\sqrt{\Sigma_{i=1}{d_{i}}^2}}} $$

実装すると以下のようになります。<br>
`np.dot()`は内積を表し、`np.linalg.norm`はベクトルのノルム（ベクトルの長さ）を表しています。

```python 
import numpy as np

def cosine_similarity(v1, v2):
    cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
    return cos_sim
```

#### 問題

- `"リンゴ リンゴ", "リンゴ ゴリラ", "ゴリラ ラッパ"`という文の類似度を計算して出力してみましょう。

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

docs = np.array([
    "リンゴ リンゴ", "リンゴ ゴリラ", "ゴリラ ラッパ"
])
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u"(?u)\\b\\w+\\b")
vecs = vectorizer.fit_transform(docs)
vecs = vecs.toarray()

# cos類似度を求める関数を定義してください。
def cosine_similarity(v1, v2):
    cos_sim = 
    return


# 類似度を比較してみましょう。
print("%1.3F" % cosine_similarity(vecs[0], vecs[1]))
print("%1.3F" % cosine_similarity(vecs[0], vecs[2]))
print("%1.3F" % cosine_similarity(vecs[1], vecs[2]))

#### ヒント

- `np.dot()`で二つのベクトルの内積を計算することができ、`np.linalg.norm`でベクトルのノルム（ベクトルの長さ）を計算することができます。

#### 解答例

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

docs = np.array([
    "リンゴ リンゴ", "リンゴ ゴリラ", "ゴリラ ラッパ"
])
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u"(?u)\\b\\w+\\b")
vecs = vectorizer.fit_transform(docs)
vecs = vecs.toarray()

# cos類似度を求める関数を定義してください。
def cosine_similarity(v1, v2):
    cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))
    return cos_sim

# 類似度を比較してみましょう。
print("%1.3F" % cosine_similarity(vecs[0], vecs[1]))
print("%1.3F" % cosine_similarity(vecs[0], vecs[2]))
print("%1.3F" % cosine_similarity(vecs[1], vecs[2]))

***

## 2.2 単語のベクトル表現

### 2.2.1 Word2Vec

前節では文書をベクトル表現として表していましたが、今回は単語をベクトル化します。<br>
単語をベクトルで表現すると、単語の意味の近さの数値化や同義語の探索などが行えます。<br>

最近の研究により、新しく生まれた単語をベクトル化するツールに <b style='color: #AA0000'>Word2Vec</b>というものがあります。  
簡潔に述べると **単語の意味や文法を捉えるために単語をベクトル表現化して次元を圧縮するツール** です。  
日本人が日常的に使う語彙数は数万から数十万といわれていますが、Word2Vecでは各単語を200次元程度のベクトルとして表現できます。 

Word2Vecを用いると単語と単語の関係性を簡単に表現でき、<br>
`「王様」 - 「男」+ 「女」 = 「女王」` <br>
`「パリ」 - 「フランス」 + 「日本」 = 「東京」` <br>
のような **単語同士の** 演算も可能となります。  
 
ここでは **Word2Vec** に「livedoor newsコーパス」というニュース記事の文書データを与え、単語の関係性を学習させていきます。

これから、 **Word2Vec** を用いて"男"という単語と関連性が高い文字列を調べていきます。<br>
フローは以下のようになります。対応するセッション名も同時に記しています。<br>

> 1. ニュース記事をテキストコーパスとして取り出し、文章とカテゴリーに分ける。 : 「globモジュール」、「with文」、「コーパスの呼び出し」
> 2. 取り出した文書を品詞ごとに分割し、リストにする。: 「janome(3)」
> 3. Word2Vecでモデルを生成。: 「Word2Vec（実装）」
> 4. 男との関連性が高い語を調べる。: 「Word2Vec（実装）」

#### 問題

- 以下の空欄に当てはまる単語の組み合わせを選択してください。
- 文章をベクトルで表現する方法を「」と言います。また単語をベクトルで表現するツールとして「」があげられます。

- 「カウント表現」、「Word2Vec」
- 「BOW」、「CBOW」
- 「BOW」、「Word2Vec」
- 「tf-idf」、「CBOW」

#### ヒント

- 以下を参考にしましょう。
>1. カウント表現：文書中の各単語の出現数
>2. BOW：文書のベクトル表現
>3. Word2Vec：単語をベクトル化するツール
>4. tf-idf：文章中の各単語の重み情報を扱う方法

#### 解答

「BOW」、「Word2Vec」

***

### 2.2.2 globモジュール

<b style='color: #AA0000'>globモジュール</b>はファイルやディレクトリを操作するときに便利なモジュールであり、正規表現を用いてパスを指定できます。

> - **ファイル**：文書、写真、音楽など、ユーザーが操作・管理する情報の最小単位。
> - **ディレクトリ**：ファイルをまとめる入れ物のこと。
> - **パス**：コンピュータ上でファイルやディレクトリの場所のこと。

globモジュールが **osモジュール** （基本的にファイルやディレクトリを操作するときに用いるモジュール）と異なる点は、<br>
**特殊な文字や文字列を用いて賢くファイルを検索できる点** です。<br>
例えば、アスタリスク **`*`** を用いて以下の例のように記述すると、 testディレクトリにあるtxtファイルを全て表示させることができます。  
```python
import glob

lis = glob.glob("test/*.txt")
print(lis)
# 出力結果
["test/sample.txt", "test/sample1.txt", "test/sample2.txt"]
```

また、以下の例のようにして特殊な文字列を用いることができます。この例では、testディレクトリ以下のsample(数字).txtファイルを全て表示させることができます。
```python
import glob

lis = glob.glob("test/sample[0-9].txt")
print(lis)
# 出力結果
["test/sample1.txt", "test/sample2.txt"]
```

#### 問題

- Aidemyが提供するコードの実行環境には、livedoor newsコーパスの大量のニュース記事データがカテゴリーごとに保存された、textディレクトリが用意されています。
globモジュールを用いて`text/sports-watch`ディレクトリの中にあるテキストファイルのファイル名のリストを取得し、リストのはじめの3つの要素を出力してください。

In [ ]:
import glob

# text/sports-watchの中にあるファイルを表示してください
lis = 
print(lis[0:3])

#### ヒント

- パス以下の全てのテキストファイルを表示させたいときは、`glob.glob("パス/*.txt")`としましょう。

#### 解答例

In [ ]:
import glob

# text/sports-watchの中にあるファイルを表示してください
lis = glob.glob("text/sports-watch/*.txt")
print(lis[0:3])

***

### 2.2.3 with文

通常ファイルを読み込む際には、`open()`でファイルを開き、`read()`等でファイルを読み込み、`close()`でファイルを閉じます。

下の例の`open()`の第二引数`"r"`はファイルを開くときのモードを指定するもので、この場合だとreadの意で読み込み専用であることを示しています。他にも、例えば書き込み専用ならwriteの意で`"w"`と指定したりします。
```python
f = open("a.text", "r", encoding="utf-8")
data = f.read()
f.close()
```
しかしこの表記の仕方だと、`close()`を書き忘れてしまったり、途中でエラーが発生しファイルが閉じられず、メモリが無駄に占有される恐れがあります。

そこで代わりに <b style='color: #AA0000'>with文</b>を用います。 **with文**　を用いると、ファイルが自動的に`close()`されたり、ファイルを開いている際にエラーが発生しても適切な例外処理が自動的に行われるので、とても便利です。

 **with文を用いた場合** 、以下のようにしてファイルを開きます。<br>
 この例では、最もポピュラーな **文字コード** `UTF-8`でファイルを出力しています。文字コードとは、コンピュータ上で文字を扱うために個々の文字に割り振られた番号のことです。 
```python
with open("a.text", "r", encoding="utf-8") as f:
```
with文中で、例えば
```python
data = f.read()
```
等でファイルの読み込みを行います。`read()`を使うと、ファイル内のデータ全てを文字列として読み込みます。

#### 問題

- `with`文を用いて`text/sports-watch`上にある`LICENSE.txt`を開き、出力してください。

In [ ]:
# with文を用いてtext/sports-watch上にあるLICENSE.txtを開き、出力してください



#### ヒント

```python
with open("a.text", "r", encoding="utf-8") as f:
```
でテキストを開けます。

#### 解答例

In [ ]:
# with文を用いてtext/sports-watch上にあるLICENSE.txtを開き、出力してください
with open("text/sports-watch/LICENSE.txt", "r", encoding="utf-8") as f:
    print(f.read())

***

### 2.2.4 コーパスの取り出し

<b style='color: #AA0000'>コーパス</b>とは、 **文書または音声データにある種の情報を与えたデータ** のことでした。

**livedoor newsコーパス** とは、ダウンロード元によると以下のようなデータとされています。
```python
NHN Japan株式会社が運営する「livedoor ニュース」のうち、クリエイティブ・コモンズライセンスが適用されるニュース記事を収集し、可能な限りHTMLタグを取り除いて作成してあるものです。
通常必要なデータの整形作業が必要ないものです。
```

Aidemyが提供するコードの実行環境には、livedoor newsコーパスの大量のニュース記事データが保存された、5050_nlp_dataディレクトリが用意されています。5050_nlp_dataディレクトリの中には9つのカテゴリーのディレクトリがあり、それぞれのディレクトリの中にニュース記事データのtxtファイルが入っています。以下のコードでtextディレクトリや、カテゴリーごとのディレクトリの内容を確認できます。
```python
glob.glob("./5050_nlp_data/*")
glob.glob("./text/sports-watch/*")
```

今回は、livedoor newsコーパスをテキストコーパスとして取り出し、文章とカテゴリーに分けて利用します。
以下のように表記することで、コーパスを取り出して分類することができます。<br>
一見複雑ですが、内容はとても単純です。

`splitlines()`は文字列を改行部分で分解し、各行からなるリストを返す組み込み関数です。
```python
import glob

def load_livedoor_news_corpus():
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = [] # 全ての記事の文章をここに格納します。
    labels = [] # docsに格納される記事の1〜9のカテゴリーを、ラベルとして扱います。

    # 全てのカテゴリーのディレクトリについて実行します。
    for c_name, c_id in category.items():
        # {c_name}にcategory.items()から取得したカテゴリー名c_nameをformatメソッドを用いて埋め込みます。
        files = glob.glob("./5050_nlp_data/{c_name}/{c_name}*.txt".format(c_name=c_name))
        # カテゴリーに属するファイル数（記事の数）を表示します。
        print("category: ", c_name, ", ",  len(files))
        # 各記事について、URL、 日付、タイトル、 本文の情報を以下のようにして取得します。
        for file in files:
            # with文を用いるため、close()は不要です。
            with open(file, "r", encoding="utf-8") as f:
                # 改行文字で分割
                lines = f.read().splitlines()
                # 分割すると0番目にurl, 1番目に日付、2番目にタイトル、3番目以降に記事本文が記載されています。
                url = lines[0]  
                datetime = lines[1]  
                subject = lines[2]
                # 記事中の本文を1行にまとめてしまいます。
                body = "".join(lines[3:])
                # タイトルと本文をまとめてしまいます。
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels
    
# 全ての記事の文章データとそのラベル（カテゴリー）を取得します。
docs, labels = load_livedoor_news_corpus()
``` 

#### 問題

- 説明文のコードを写して実行しましょう。

In [ ]:
import glob

def load_livedoor_news_corpus():
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []
    
    # 以下に説明文のコードの該当箇所を写してください
    #-----------------------------------
    
    
    
    
    
    
    
    
    #-----------------------------------

docs, labels = load_livedoor_news_corpus()

print("\nlabel: ", labels[0], "\ndocs:\n", docs[0])
print("\nlabel: ", labels[1000], "\ndocs:\n", docs[1000])

#### ヒント

- 最後の`print`文では、2つの記事の全文と記事のラベル（カテゴリー）が表示されます。

#### 解答例

In [ ]:
import glob

def load_livedoor_news_corpus():
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []
    
    
    # 例のコードを写してください。
    for c_name, c_id in category.items():
        files = glob.glob("./5050_nlp_data/{c_name}/{c_name}*.txt".format(c_name=c_name))
        text = ""
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 
                url = lines[0]  
                datetime = lines[1]  
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels

docs, labels = load_livedoor_news_corpus()
print("\nlabel: ", labels[0], "\ndocs:\n", docs[0])
print("\nlabel: ", labels[1000], "\ndocs:\n", docs[1000])

***

### 2.2.5 Word2Vec（実装）

下準備はできたので、本題の **Word2Vec** を説明していきます。<br>
Word2Vecを用いる時は`gensim`モジュールから`import`します。
```python
from gensim.models import word2vec
```
学習に使用するリスト（分かち書きされた文書）を **`Word2Vec`関数** の引数とすることで、モデルを生成します。<br>

「BOW カウント」等で扱った`janome.tokenizer`を使って予め **分かち書き** を行います。
分かち書きの際、各単語について品詞を調べます。  
日本語では、「名詞、動詞、形容詞、形容動詞」以外は単語の関連性の分析に使えないので、「名詞、動詞、形容詞、形容動詞」のみの分かち書きリストを作成します。<br>


**`Word2Vec`** は以下のようにして使います。
```python
model = word2vec.Word2Vec(リスト, size=a, min_count=b, window=c)
# ただし、a, b, cは数字
```
`Word2Vec`のよく使う引数は主に以下です。
> - **size** ：ベクトルの次元数。
> - **window** ：この数の前後の単語を、関連性のある単語と見なして学習を行う。
> - **min_count** ：n回未満登場する単語を破棄。

適切に学習が行われた後、  `model`に対し
 **`.most_similar(positive=["単語"])`** のように`most_similar()`メソッドを用いるとその単語との類似度が高いものが出力されます。<br>

#### 問題

- `word2vec`を用いて、「男」という単語と関連性の高い単語を出力してください。`word2vec.Word2Vec`の引数に関して、リストには`[sentences]`を指定し、`size=100`, `min_count=20`, `window=15`としてください。

In [ ]:
import glob
from janome.tokenizer import Tokenizer
from gensim.models import word2vec

# livedoor newsの読み込みと分類
def load_livedoor_news_corpus():
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./5050_nlp_data/{c_name}/{c_name}*.txt".format(c_name=c_name))

        text = ""
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 

                #1,2行目に書いたあるURLと時間は関係ないので取り除きます。
                url = lines[0]  
                datetime = lines[1]  
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels

# 品詞を取り出し「名詞、動詞、形容詞、形容動詞」のリスト作成
def tokenize(text):
    t = Tokenizer()
    tokens = t.tokenize(",".join(text))
    word = []
    for token in tokens:
        part_of_speech = token.part_of_speech.split(",")[0]
 
        if part_of_speech in ["名詞", "動詞", "形容詞", "形容動詞"]:
            word.append(token.surface)            
    return word

# ラベルと文章に分類
docs, labels = load_livedoor_news_corpus()
sentences = tokenize(docs[0:100])  # データ量が多いため制限している

# 以下に回答を作成してください
#word2vec.Word2Vecの引数に関して、size=100, min_count=20, window=15としてください
model = 
print()

#### ヒント

```python
model = word2vec.Word2Vec(リスト, size=100, min_count=20, window=15)
print(model.most_similar(positive=["単語"]))
```

#### 解答例

In [ ]:
import glob
from janome.tokenizer import Tokenizer
from gensim.models import word2vec

# livedoor newsの読み込みと分類
def load_livedoor_news_corpus():
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./5050_nlp_data/{c_name}/{c_name}*.txt".format(c_name=c_name))

        text = ""
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 

                #1,2行目に書いたあるURLと時間は関係ないので取り除きます。
                url = lines[0]  
                datetime = lines[1]  
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels

# 品詞を取り出し「名詞、動詞、形容詞、形容動詞」のリスト作成
def tokenize(text):
    t = Tokenizer()
    tokens = t.tokenize(",".join(text))
    word = []
    for token in tokens:
        part_of_speech = token.part_of_speech.split(",")[0]
 
        if part_of_speech in ["名詞", "動詞", "形容詞", "形容動詞"]:
            word.append(token.surface)            
    return word

# ラベルと文章に分類
docs, labels = load_livedoor_news_corpus()
sentences = tokenize(docs[0:100])  # データ量が多いため制限している
# 以下に回答を作成してください
#word2vec.Word2Vecの引数に関して、size=100, min_count=20, window=15としてください
model = word2vec.Word2Vec([sentences], size=100, min_count=20, window=15)
print(model.most_similar(positive=["男"]))

## 2.3 Doc2Vec

### 2.3.1 Doc2Vec（1）

<b style='color: #AA0000'>Doc2Vec</b>は、Word2Vecを応用した **文章をベクトル化する技術** です。<br>
「文書のベクトル表現」にてBOWで文章のベクトル化を勉強しましたが、BOWとの大きな違いは **文の語順** も特徴として考慮に入れることができる点です。

文書のベクトル表現にて学んだBOWの欠点をおさらいすると、以下のようになります。

- 単語の語順情報がない
- 単語の意味の表現が苦手

この二点の欠点をDoc2Vecは補っています。


#### 問題

- 選択肢から正しいものを選択してください。

- BOWは、単語の語順を考慮している。
- Word2Vecは、単語の語順を考慮している。
- Doc2Vecは、単語の意味の表現が苦手である。
- Doc2Vecは、単語の語順を考慮している。

#### ヒント

- BOW,Word2Vecは文書のベクトル表現やWord2Vecの章で学びました。
- Doc2VecはBOWの、単語の語順情報がない、単語の意味の表現が苦手、という点を補ったものです。

#### 解答

Doc2Vecは、単語の語順を考慮している。

***

### 2.3.2 Doc2Vec（2）

<b style='color: #AA0000'>Doc2Vec</b>を実装していきます。<br>
「コーパスの取り出し」にて作成したlivedoor newsコーパスの`docs[0],docs[1],docs[2],docs[3]`の類似度を比較します。

フローは以下のようになります。
1. 分かち書き
1. **`TaggedDocument`** クラスのインスタンスを作成
1. **`Doc2Vec`** でモデルの生成
1. 類似度の出力
***
**ポイント**

1.<br>
文章を`janome`の`Tokenizer`を用い、分かち書きにします。

2.<br>
`TaggedDocument`の引数に`words="分かち書きされた各要素", tags=["タグ"]`を与えると、 **TaggedDocument** クラスのインスタンスを作成できます。<br>
タグは文書のidのようなものです。<br>
`TaggedDocument`をリストに格納し、これをDoc2Vecに渡します。

3.<br>
モデルの学習は以下のように記述します。
```python
model = Doc2Vec(documents=リスト, min_count=1)
```
`min_count`:最低この回数出現した単語のみを学習に使用<br>

4.<br>
類似度の出力は以下のように記述します。
```python
for i in range(4):
    print(model.docvecs.most_similar("d"+str(i)))
```


問題にDoc2Vecの例を載せているので、解きながら使い方を覚えて行きましょう。

#### 問題

- 次のコードのモデル学習の部分を埋めて、livedoor newsコーパスの`docs[0],docs[1],docs[2],docs[3]`の類似度を比較してください。

In [ ]:
import glob
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from janome.tokenizer import Tokenizer

# livedoor newsの読み込みと分類
def load_livedoor_news_corpus():
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./5050_nlp_data/{c_name}/{c_name}*.txt".format(c_name=c_name))

        text = ""
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 

                #1,2行目に書いたあるURLと時間は関係ないので取り除きます。
                url = lines[0]  
                datetime = lines[1]  
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels
docs, labels = load_livedoor_news_corpus()

# Doc2Vecの処理
token = [] # 各docsの分かち書きした結果を格納するリストです
training_docs = [] # TaggedDocumentを格納するリストです
for i in range(4):
    
    # docs[i] を分かち書きして、tokenに格納します
    t = Tokenizer() 
    token.append(t.tokenize(docs[i], wakati=True))
    
    # TaggedDocument クラスのインスタンスを作成して、結果をtraining_docsに格納します
    # タグは "d番号"とします
    training_docs.append(TaggedDocument(words=token[i], tags=["d" + str(i)]))

# 以下に回答を作成してください
model = 

for i in range(4):
    print(model.docvecs.most_similar("d"+str(i)))



#### ヒント

- `script.py`にはモデル学習の記述が抜けています。
```python
model = Doc2Vec(documents=リスト, min_count=1)
```

#### 解答例

In [ ]:
import glob
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from janome.tokenizer import Tokenizer

# livedoor newsの読み込みと分類
def load_livedoor_news_corpus():
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./5050_nlp_data/{c_name}/{c_name}*.txt".format(c_name=c_name))

        text = ""
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 

                #1,2行目に書いたあるURLと時間は関係ないので取り除きます。
                url = lines[0]  
                datetime = lines[1]  
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels
docs, labels = load_livedoor_news_corpus()

# Doc2Vecの処理
token = [] # 各docsの分かち書きした結果を格納するリストです
training_docs = [] # TaggedDocumentを格納するリストです
for i in range(4):
    
    # docs[i] を分かち書きして、tokenに格納します
    t = Tokenizer() 
    token.append(t.tokenize(docs[i], wakati=True))
    
    # TaggedDocument クラスのインスタンスを作成して、結果をtraining_docsに格納します
    # タグは "d番号"とします
    training_docs.append(TaggedDocument(words=token[i], tags=["d" + str(i)]))

# 以下に回答を作成してください
model = Doc2Vec(documents=training_docs, min_count=1)

for i in range(4):
    print(model.docvecs.most_similar("d"+str(i)))



***

## 2.4 日本語テキストの分類

### 2.4.1 日本語テキストの分類

chapter1, chapter2で学んできたことを用いて <b style='color: #AA0000'>**日本語テキストのカテゴリをランダムフォレスト**</b>で分類します。<br>
ここでもlivedoor newsを用います。ランダムフォレストに与えるデータはベクトル表現化したニュース記事で、9種類のカテゴリーに分類します。   
記事をベクトルで表すことにより、教師あり学習で学んだ方法をそのまま適用して記事の分類が行えます。 

このチャプターの学習フローは以下のようになります。

1. livedoor newsの読み込みと分類：「コーパスの取り出し」
2. データをトレイニングデータとテストデータに分割する：機械学習概論の「ホールドアウト法の理論と実践」
3. tf-idfでトレイニングデータとテストデータをベクトル化する：「BOW tf-idfによる重み付け（実装）」、「fit関数」
4. ランダムフォレストで学習：教師あり分類の「ランダムフォレスト」
5. 実装：「コーパスのカテゴリをランダムフォレストで実装」
6. 精度を上げる：「精度をあげる」

#### 問題

以下の選択肢から正しい順番に並んでいるものを選択してください。<br>
1. 機械学習手法でデータを学習（基準の取得）
2. データ収集
3. テストデータで性能をテスト
4. データクレンジング

- 2  -> 3 -> 1 -> 4
- 2  -> 1 -> 3 -> 4
- 2  -> 3 -> 4 -> 1
- 2 -> 4 -> 1 -> 3
- 2 -> 1 -> 4 -> 3

#### ヒント

- データクレンジングはデータの前処理になります。

#### 解答

- 2 -> 4 -> 1 -> 3

***

### 2.4.2 fit関数

`scikit-learn` の変換系クラス(StandardScaler、Normalizer、TfidfVectorizer など)には、 **`fit()`, `fit_transform()`, `transform()`** などの関数があります。<br>

**`fit()`関数** ：渡されたデータの統計（最大値、最小値、平均、など）を取得して、メモリに保存。<br>
**`transform()`関数** ：`fit()`で取得した情報を用いてデータを書き換える。<br>
**`fit_transform()`関数** ：`fit()`の後に`transform()`を実施する。

 **`fit()関数`** はトレーニングデータセットからパラメーターを学習するために使用され、 **`tranform()関数`** は　学習したパラメーターに基づいてデータが再形成されます。<br>
つまり、トレーニングデータの場合は **`fit_transform関数`** を用い、<br>テストデータの場合は,トレーニングデータの **`fit()`** の結果に基づくので、  **`transform()関数`** を行う必要があります。

<img src="https://aidemyexcontentspic.blob.core.windows.net/contents-pic/5050_nlp/fit_func.png" width=500>

#### 問題

- 以下の選択肢から正しいものを選んでください。

- テストデータに `fit_transform()` を用いた。
- テストデータに `fit()` したものを `transform()` した。
- トレーニングデータに `fit_transform()` を用いた。

#### ヒント

- fit()させるのはトレーニングデータです。テストデータにはfitさせてはいけません。

#### 解答

- トレーニングデータに `fit_transform()` を用いた。

***

### 2.4.3 コーパスのカテゴリをランダムフォレストで実装

それではこれまで学んできたことを用いて、 <b style='color: #AA0000'>livedoornewsコーパスのカテゴリをランダムフォレストで分類</b>しましょう。

「日本語テキストの分類」でも書いたようにフローは以下のようになります。

1. livedoor newsの読み込みと分類：「コーパスの取り出し」
2. データをトレイニングデータとテストデータに分割する：機械学習概論の「ホールドアウト法の理論と実践」
3. tf-idfでトレイニングデータとテストデータをベクトル化する：「BOW tf-idfによる重み付け（実装）」、「fit関数」
4. ランダムフォレストで学習：教師あり分類の「ランダムフォレスト」
5. 実装：「コーパスのカテゴリをランダムフォレストで実装」
6. 精度を上げる：「精度をあげる」


#### 問題

- livedoornewsコーパスのカテゴリをランダムフォレストで分類しましょう。
- `train_data`と`test_data`をtf-idfでベクトル化した値をそれぞれ`train_matrix`と`test_matrix`に代入しましょう。

In [ ]:
import glob
import random
from sklearn.model_selection import train_test_split
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

def load_livedoor_news_corpus():
# livedoor newsの読み込みと分類(自然言語処理基礎2.2.4)
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./5050_nlp_data/{c_name}/{c_name}*.txt".format(c_name=c_name))

        text = ""
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 
                url = lines[0]  
                datetime = lines[1]  
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels

docs, labels = load_livedoor_news_corpus()

# データをトレイニングデータとテストデータに分割(機械学習概論2.2.2)
train_data, test_data, train_labels, test_labels = train_test_split(docs, labels, test_size=0.2, random_state=0)

# tf-idfでトレイニングデータとテストデータをベクトル化(自然言語処理基礎2.1.4, 2.4.2)
# 以下に回答を作成してください
vectorizer = 
train_matrix = 
test_matrix = 

# ランダムフォレストで学習
clf = RandomForestClassifier(n_estimators=2)
clf.fit(train_matrix, train_labels)

# 精度の出力
print(clf.score(train_matrix, train_labels))
print(clf.score(test_matrix, test_labels))

#### ヒント

- トレーニングデータには`fit_transform`を、テストデータには`transform`を使用しましょう。
- わからない部分は書いてあるセッションを参照して復習をしましょう。

#### 解答例

In [ ]:
import glob
import random
from sklearn.model_selection import train_test_split
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

def load_livedoor_news_corpus():
# livedoor newsの読み込みと分類(自然言語処理基礎2.2.4)
    category = {
        "dokujo-tsushin": 1,
        "it-life-hack":2,
        "kaden-channel": 3,
        "livedoor-homme": 4,
        "movie-enter": 5,
        "peachy": 6,
        "smax": 7,
        "sports-watch": 8,
        "topic-news":9
    }
    docs  = []
    labels = []

    for c_name, c_id in category.items():
        files = glob.glob("./5050_nlp_data/{c_name}/{c_name}*.txt".format(c_name=c_name))

        text = ""
        for file in files:
            with open(file, "r", encoding="utf-8") as f:
                lines = f.read().splitlines() 
                url = lines[0]  
                datetime = lines[1]  
                subject = lines[2]
                body = "".join(lines[3:])
                text = subject + body

            docs.append(text)
            labels.append(c_id)

    return docs, labels

docs, labels = load_livedoor_news_corpus()

# データをトレイニングデータとテストデータに分割(機械学習概論2.2.2)
train_data, test_data, train_labels, test_labels = train_test_split(docs, labels, test_size=0.2, random_state=0)

# tf-idfでトレイニングデータとテストデータをベクトル化(自然言語処理基礎2.1.4, 2.4.2)
# 以下に回答を作成してください
vectorizer = TfidfVectorizer()
train_matrix = vectorizer.fit_transform(train_data)
test_matrix = vectorizer.transform(test_data)

# ランダムフォレストで学習
clf = RandomForestClassifier(n_estimators=2)
clf.fit(train_matrix, train_labels)

# 精度の出力
print(clf.score(train_matrix, train_labels))
print(clf.score(test_matrix, test_labels))

***

### 2.4.4 精度を上げる

「コーパスのカテゴリをランダムフォレストで実装」で実装したプログラムの精度を上げる作業をしたいと思います。
 **`TfidfVectorizer()`** のパラメーターに **`tokenizer=関数`** を設定すると、指定した関数でテキストを分割することができます。<br>
例えば、以下の関数を **`tokenizer=`** の引数とすると、「名詞、動詞、形容詞、形容動詞」のみの分割されたテキストを用います。<br>
そのため、分析に必要ない助詞・助動詞等がないので精度が上がる事になります。
```python
from janome.tokenizer import Tokenizer

def tokenize(text):
    t=Tokenizer()
    tokens = t.tokenize(",".join(text))
    noun = []
    for token in tokens:
    # 品詞を取り出し
        partOfSpeech = token.part_of_speech.split(",")[0]
 
        if partOfSpeech == "名詞":
            noun.append(token.surface)        
        if partOfSpeech == "動詞":        
            noun.append(token.surface)
        if partOfSpeech == "形容詞":
            noun.append(token.surface)        
        if partOfSpeech == "形容動詞":        
            noun.append(token.surface)            
    return noun
```

#### 問題

- 以下の選択肢から **最も精度が高くなると思われるもの** を選択してください。

- 名詞のみで分割したテキスト
- 動詞のみで分割したテキスト
- 形容詞のみで分割したテキスト
- 形容動詞のみで分割したテキスト

#### ヒント

- 最も文書の特徴がでるキーワードの品詞を考えてください。

#### 解答

- 名詞のみで分割したテキスト

***